In [ ]:

import os

os.environ["KAGGLE_USERNAME"] = "manith04"
os.environ["KAGGLE_KEY"] = "KGAT_3281ee8bdd9e4c418ae9f767420667f3"

import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Initialize API
api = KaggleApi()
api.authenticate()

# Dataset ID
dataset = "manith04/rt-image"

# Download directory
download_dir = "./rt-image"
os.makedirs(download_dir, exist_ok=True)

# Download and unzip
api.dataset_download_files(dataset, path=download_dir, unzip=True)

print(f"Dataset downloaded and extracted to {download_dir}")


Dataset URL: https://www.kaggle.com/datasets/manith04/rt-image
Dataset downloaded and extracted to ./rt-image


In [ ]:
%cd /content/rt-image/MVSA

/content/rt-image/MVSA


In [ ]:
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset
from PIL import Image, ImageFile
import pandas as pd
import os
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm

ImageFile.LOAD_TRUNCATED_IMAGES = True

# -------------------------
# 1. Load dataset
# -------------------------
df = pd.read_csv("mvsa_image_soft_labels.txt", sep="\t")  # adjust sep if needed
df['ID'] = df['ID'].astype(int).astype(str)

# Stratified split
df_train, df_temp = train_test_split(df, test_size=0.2, random_state=42, stratify=df.iloc[:,1])
df_val, df_test  = train_test_split(df_temp, test_size=0.5, random_state=42, stratify=df_temp.iloc[:,1])

print(f"Train: {len(df_train)}, Val: {len(df_val)}, Test: {len(df_test)}")

# -------------------------
# 2. Dataset class (robust)
# -------------------------
class SafeImageDataset(Dataset):
    def __init__(self, dataframe, image_dir):
        self.image_dir = image_dir
        self.label_map = {'negative':0, 'neutral':1, 'positive':2}
        self.data = dataframe.reset_index(drop=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        try:
            row = self.data.iloc[idx]
            img_id = str(row['ID'])
            img_path = None
            for ext in ["png", "jpg", "jpeg"]:
                temp_path = os.path.join(self.image_dir, f"{img_id}.{ext}")
                if os.path.exists(temp_path):
                    img_path = temp_path
                    break

            if img_path is None:
                raise FileNotFoundError

            image = Image.open(img_path).convert("L")  # grayscale for SIFT
            image = np.array(image)

            label_cell = row.iloc[1]
            label_str = str(label_cell)
            if ',' not in label_str:
                image_label = 'neutral'
            else:
                _, image_label = label_str.split(',')

            label = self.label_map.get(image_label, 1)

        except Exception:
            # On error, return blank image and neutral label
            image = np.zeros((456,456), dtype=np.uint8)
            label = 1

        return image, label

# -------------------------
# 3. Load image IDs and labels (no full images in memory)
# -------------------------
train_dataset = SafeImageDataset(df_train, "data")
val_dataset   = SafeImageDataset(df_val, "data")
test_dataset  = SafeImageDataset(df_test, "data")

# -------------------------
# 4. SIFT + MiniBatchKMeans (fit in batches)
# -------------------------
sift = cv2.SIFT_create()
VOCAB_SIZE = 200
kmeans = MiniBatchKMeans(n_clusters=VOCAB_SIZE, batch_size=100, verbose=1, random_state=42)

# Batch-wise KMeans fitting
BATCH_SIZE = 50
print("Fitting KMeans in batches...")
for start in range(0, len(train_dataset), BATCH_SIZE):
    batch_descs = []
    for i in range(start, min(start+BATCH_SIZE, len(train_dataset))):
        img, _ = train_dataset[i]
        kp, desc = sift.detectAndCompute(img, None)
        if desc is not None:
            batch_descs.append(desc)
    if batch_descs:
        batch_descs = np.vstack(batch_descs)
        kmeans.partial_fit(batch_descs)

# -------------------------
# 5. Function to compute BoVW histograms batch-wise
# -------------------------
def compute_histograms(dataset, batch_size=50):
    features, labels = [], []
    for start in range(0, len(dataset), batch_size):
        for i in range(start, min(start+batch_size, len(dataset))):
            img, lbl = dataset[i]
            kp, desc = sift.detectAndCompute(img, None)
            if desc is None:
                desc = np.zeros((1,128))
            words = kmeans.predict(desc)
            hist, _ = np.histogram(words, bins=np.arange(VOCAB_SIZE+1))
            features.append(hist)
            labels.append(lbl)
    return np.array(features), np.array(labels)

# -------------------------
# 6. Compute BoVW histograms
# -------------------------
print("Computing BoVW histograms for train set...")
X_train, y_train = compute_histograms(train_dataset)
print("Computing BoVW histograms for val set...")
X_val, y_val     = compute_histograms(val_dataset)
print("Computing BoVW histograms for test set...")
X_test, y_test   = compute_histograms(test_dataset)

# -------------------------
# 7. Normalize
# -------------------------
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val   = scaler.transform(X_val)
X_test  = scaler.transform(X_test)

# -------------------------
# 8. Train SVM
# -------------------------
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

# -------------------------
# 9. Evaluate
# -------------------------
y_val_pred = svm.predict(X_val)
val_acc = accuracy_score(y_val, y_val_pred)
print("Validation Accuracy:", val_acc)

y_test_pred = svm.predict(X_test)
test_acc = accuracy_score(y_test, y_test_pred)
print("Test Accuracy:", test_acc)


Train: 15680, Val: 1960, Test: 1960
Fitting KMeans in batches...
